In [2]:
!pip install -U transformers==3.0.0
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git

     |████████████████████████████████| 757kB 6.3MB/s 
     |████████████████████████████████| 1.2MB 12.5MB/s 
     |████████████████████████████████| 3.0MB 54.8MB/s 
     |████████████████████████████████| 890kB 56.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0cc75aad70f61b9bea7bc0542403fb4603900fbd35a4b337fbd51880b36b4437
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'question_generation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressi

In [3]:
%cd question_generation

/content/question_generation


In [4]:
from pipelines import pipeline
import json

In [5]:
def getPassages(filename):
    with open(filename) as file:
        data = json.load(file)
    passages = list()
    for i in data["vertices"]["paragraphs"]:
      passages.append(i["text"])
    return passages

In [10]:
def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

In [20]:
def getTokenizedAnswers(context, answers):
  start_word = answers[:2]
  # print("context:",context,"answer:", answers)
  start_answer = context.index(answers)
  end_answer = start_answer + len(answers)
  return {
    "answer": answers,
    "start_answer": start_answer,
    "end_answer": end_answer
  }

In [8]:
passages = getPassages("../handbook_graph.json")

In [9]:
qg = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [21]:
qaList = list()
count = 0
questions_dictionary = dict()
id = 0
for p in passages:
  try:
    qaList = qg(p)
    questions_dictionary[id] = {
        'qas':dict(),
        'context':p
    }
    qid = 0
    qa = questions_dictionary[id]["qas"]
    for questions in qaList:
      qa[qid] = dict()
      answers = getTokenizedAnswers(p, questions["answer"])
      qa[qid]["question"] = questions["question"]
      qa[qid]["answer"] = answers
      qid += 1
    id += 1
  except:
    count += 1
    print('error for para:', p)
    continue
print(len(qaList))

error for para: At the beginning of each semester, a student shall register for the courses a student or she wishes to take in that semester. A student shall normally be allowed to register for a course only if A student or she has passed all the necessary pre-requisites for a course. 
error for para: Registration will generally be done just before the start of classes in a semester. Pre- registration for a semester can be done toward the end of the previous semester. Students will be informed about these dates and other details every semester. 
error for para: Students can avail underload below 12 credits (as low as 4 credits) only in one semester during Students entire btech program. For this, prior permission is required to be taken before the last date of add or drop of that particular semester. Students wanting to avail internship (industrial, academic, research, etc.) without taking semester leave can do so with prior approval of the DOAA in only one semester of Students wanting 

In [22]:
write_json(questions_dictionary, "training_set_nueral.json")